In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute

GRAPH_NAME = "Qiskit original"
G = nx.Graph()
G.add_nodes_from([0, 1, 2, 3])
G.add_edges_from([(0, 1), (1, 2), (2, 3), (3, 0)])
nx.draw(G, with_labels=True, alpha=0.8, node_size=500)

In [ ]:
create_qaoa_circ(G, [1.0, 1.0]).draw(filename="../latex/img/circuits/qiskit/qiskit_circuit_p-1_1.0.png")
create_qaoa_circ(G, [2.0, 2.0]).draw(filename="../latex/img/circuits/qiskit/qiskit_circuit_p-1_2.0.png")
create_qaoa_circ(G, [1.0, 1.0, 1.0, 1.0]).draw(filename="../latex/img/circuits/qiskit/qiskit_circuit_p-2_1.0.png")
create_qaoa_circ(G, [2.0, 2.0, 2.0, 2.0]).draw(filename="../latex/img/circuits/qiskit/qiskit_circuit_p-2_2.0.png")
create_qaoa_circ(G, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]).draw(filename="../latex/img/circuits/qiskit/qiskit_circuit_p-3_1.0.png")
create_qaoa_circ(G, [2.0, 2.0, 2.0, 2.0, 2.0, 2.0]).draw(filename="../latex/img/circuits/qiskit/qiskit_circuit_p-3_2.0.png")

In [ ]:
def maxcut_obj(x, G):
    obj = 0
    for i, j in G.edges():
        if x[i] != x[j]:
            obj -= 1
    return obj

def compute_expectation(counts, G):
    avg = 0
    sum_count = 0

    for bitstr, count in counts.items():
        obj = maxcut_obj(bitstr[::-1], G)
        avg += obj * count
        sum_count += count

    return avg/sum_count

def create_qaoa_circ(G, theta):
    nqubits = len(G.nodes())
    p = len(theta)//2 # num layers
    qc = QuantumCircuit(nqubits)

    beta = theta[:p]
    gamma = theta[p:]

    # |v0>
    for i in range(0, nqubits):
        qc.h(i)

    for irep in range(0, p):
        qc.barrier()

        # Hp
        for pair in list(G.edges()):
            qc.rzz(2 * gamma[irep], pair[0], pair[1])

        qc.barrier()

        # Hm
        for i in range (0, nqubits):
            qc.rx(2 * beta[irep], i)

    qc.measure_all()

    return qc

def get_expectation(G, shots=512):
    backend = Aer.get_backend('qasm_simulator')
    backend.shots = shots

    def execute_circ(theta):
        qc = create_qaoa_circ(G, theta)
        counts = backend.run(qc, nshots=512).result().get_counts()
        return compute_expectation(counts, G)

    return execute_circ

In [ ]:
from scipy.optimize import minimize

expectation = get_expectation(G)

num_layers = 1
res = minimize(expectation, [1.0, 1.0] * num_layers, method = "COBYLA")
res

In [ ]:
from qiskit.visualization import plot_histogram

backend = Aer.get_backend('aer_simulator')
backend.shots = 512

qc_res = create_qaoa_circ(G, res.x)

counts = backend.run(qc_res, seed_simulator=10).result().get_counts()
plot_histogram(counts)

In [ ]:
# Gamma function
import numpy as np
import matplotlib.pyplot as plt

def graph():
    expectation = get_expectation(G)
    x = np.arange(0.3, 6, 0.01)
    y = []
    beta = 1.0
    for gamma in x:
        y.append(expectation([beta, gamma]))

    plt.plot(x, y)
    # plt.savefig("../../latex/img/zhiqiang_grafo/modificadores_paper/zhiqiang_p_20_gamma_fun.png")
    plt.show()

graph()

In [ ]:
# Statistics
shots = 512

def max_global_statistics(num_layers=1, num_generations=1000):
    max_statistics = {}
    global_statistics = {}
    for iteration in range(0, num_generations):
        expectation = get_expectation(G)
        res = minimize(expectation, [1.0, 1.0] * num_layers, method = "COBYLA")
        qc = create_qaoa_circ(G, res.x)
        counts = backend.run(qc, shots=shots).result().get_counts()

        # Max statistics
        max_path = max(counts, key=counts.get)
        if max_path not in max_statistics:
            max_statistics[max_path] = 0
        max_statistics[max_path] += 1

        # Global statistics
        for (path, num_appearances) in counts.items():
            if path not in global_statistics:
                global_statistics[path] = 0
            global_statistics[path] += num_appearances

    for path in global_statistics:
        global_statistics[path] = global_statistics[path] / shots / num_generations  # Normalize

    for path in max_statistics:
        max_statistics[path] = max_statistics[path] / num_generations  # Normalize

    max_statistics = sorted(max_statistics.items(), key=lambda x: x[1], reverse=True)
    global_statistics = sorted(global_statistics.items(), key=lambda x: x[1], reverse=True)

    return max_statistics, global_statistics


print(GRAPH_NAME + ":\n")
interval_num_layers = (1, 3)
for p in range(interval_num_layers[0], interval_num_layers[1] + 1):
    max_st, global_st = max_global_statistics(num_layers=p, num_generations=100)
    print(f"Max statistics (p = {p}): ", str(max_st))
    print(f"Global statistics (p = {p}): ", str(global_st))
    print()

In [ ]:
!dunstify -t $((30* 1000*60)) --urgency=critical "Ejecución completada"